<a href="https://colab.research.google.com/github/Shaandili/robotics-me639-2022/blob/Shaandili/ME_639_Mini_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
#if there are any issues while viewing animation, please skip the FuncAnimation method
#instead, please copy-paste the for loop towards the end of task 1 part 1 (right before anim has been defined)
#it is in its own code cell, and can be used to view individual frames/ a crude animation
#task 1 part 1
from matplotlib import rc
rc('animation',html = 'jshtml') #to view the animation object in colab
import math as m 
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation #to create animation objects
import sys
import seaborn as sns
from scipy.integrate import odeint #to solve ordinary differential equations

In [ ]:
PI = 3.14159
m1 = int(input())
m2 = int(input())
l1 = int(input())
l2 = int(input())

In [5]:
Rmax = l1 + l2 #maximum extent of workspace
Rmin = abs(l1 - l2) #minimum extent of workspace

In [ ]:
#function to plot axes:
def initialize():
  ax = plt.axes(xlim = (-Rmax-0.1,Rmax+0.1), ylim = (-Rmax-0.1,Rmax+0.1))
#creating a figure object, specifying size
fig = plt.figure(figsize = (5,5))

In [7]:
#checks if a point is within the workspace (assuming both arms can move by 360 degrees)
def isworkspace(x,y):
  dist = m.sqrt(x**2 + y**2)  #finds point's distance from base
  if dist>Rmax or dist<Rmin:  #point should fall between the min and max extent of workspace
    return 0 #this means the point is not in the workspace
  else:
    return 1 #means the point is in the workspace

In [8]:
#just as an example, testing out the trajectory of a circle of radius = 1.5 centered at the base
angle = np.linspace(0,6.28,100) #creating an array of parameter values

x = 1.5*np.cos(angle) #array of x values using parameter
y = 1.5*np.sin(angle) #array of y values using parameter


In [9]:
#function to draw a single frame of an animation
def frame(w):
  global x,y #calls upon the arrays we created previously
  if not isworkspace(x[w],y[w]): #checks if each point in trajectory is in workspace
    print("not in workspace")
    sys.exit() #if not in workspace, prints error message and exits
  else:
    #application of first few equations:
    theta = m.acos((x[w]**2 + y[w]**2 - l1**2 - l2**2)/(2*l1*l2))
    q1 = m.atan(y[w]/x[w]) - m.atan(l2*m.sin(theta)/(l1 + l2*m.cos(theta)))
    q2 = q1 + theta
    #finding the end points of each arm - end effector points already known, x1 and y1 are end points of first arm
    x1 = l1*m.cos(q1)
    y1 = l1*m.sin(q1)
    #arrays with x and y points of base (0,0) and the end points of each arm
    X = [0,x1,x[w]]
    Y = [0,y1,y[w]]
    #for clearing the previous frame's animation and initializing the axes 
    #(however, as of now including these creates and error where we can't view the animation object at all)
    # plt.clf()
    # initialize()
    #creating an object that plots the lines for the robot's arms
    pl = plt.plot(X,Y)
    #returning the object
    return pl

In [ ]:
# a way to output the animation without using FuncAnimation - however, it gives as many animations as there are frames 
# also serves as a way to view individual frames
for i in range(100):
  plt.clf()
  initialize()
  frame(i)
  plt.pause(0.01)

In [ ]:
#creating the animation object
anim = animation.FuncAnimation(fig,frame, frames = 100, blit = True)

In [ ]:
anim #viewing the animation object

In [16]:
#task 1 part 2
#input trajectory
#trajectory would be defined by tau values, not the linear spaces here

#input torque values
tau1 = int(input())
tau2 = int(input())
#array of torque values
tau = [tau1, tau2]

t = np.linspace(0,100,100) #vector for time points

#initial conditions for q1 and q2
q10 = 0
q20 = 0
#array for initial conditions of q1 and q2
q0 = [q10, q20]
#taking initial angular velocities to be 0
Q10 = 0
Q20 = 0
#array for initial conditions of Q1 and Q2
Q0 = [Q10, Q20]

In [ ]:
#method to define 2nd order ode
# odeint(ode2) solves the 2nd order diff eq (written as first order of Q)
def ode2(Q,t):
  g = 9.81
  Q1 = Q[0]
  Q2 = Q[1]
  q1 = []
  q2 = []
  global m1,m2,l1,l2, tau1, tau2,q10,q20
  if t == 0:
    q1 = q10
    q2 = q20
  A = (m1*(l1**2)/3) + (m2*(l1**2))
  B = (m2*l1*l2/2)*np.cos(q2 - q1)
  F = (m2*(l2**2)) + (m2*(l2**2)/4)
  P = tau1 - (m2*g*l1*np.cos(q2)) - (m1*g*l1*np.cos(q2)/2) + (m2*l2*l1*Q2*(Q2 - Q1)/2)*np.sin(q2 - q1)
  T = tau2 + (m2*l1*l2/2)*Q1*(Q2 - Q1)*np.sin(q2 - q1) - (m2*g*l2/2)*np.sin(q2)
  mat1 = [[A,B],[B,F]]
  mat2 = [P,T]
  qsol = np.matmul(np.linalg.inv(mat1),mat2)
  dQ1dt = qsol[0]
  dQ2dt = qsol[1]
  return [dQ1dt, dQ2dt]

Q = odeint(ode2,Q0,t) #solution for angular velocities

In [ ]:
#method to define 1st order ode (using solutions of ode 2)
# odeint(ode1) solves the 1st order diff eq
def ode1(q,t,Q):
  q1 = q[0]
  q2 = q[1]
  dq1dt = Q[0]
  dq2dt = Q[1]
  return [dq1dt,dq2dt]

q = odeint(ode1,q0,t) #solution for joint variables as a function of time

In [ ]:
#method to define first order ode (using solutions of ode1)

def odexy(X,t,Q,q):
  x = X[0]
  y = X[1]
  cartspace = [[-1*l1*np.sin(q1), -1*l2*np.sin(q2)],[l1*np.cos(q1), l2*np.cos(q2)]]
  xsol = np.matmul(cartspace,Q)
  dxdt = xsol[0]
  dydt = xsol[1]
  return [dxdt, dydt]
  
#inital conditions for end effector point
x0 = l1+l2
y0 = 0
X0 = [x0, y0]

#finding x and y as a function of time
X = odeint(odexy,X0,t)
x = X[0]
y = X[1]

In [ ]:
#calling frame method to panimate position of robot at different instances of time
speed = animation.FuncAnimation(fig,frame, frames = 100, blit = True)

In [ ]:
speed #viewing animation

In [ ]:
#task 2
Fx = int(input())
Fy = int(input())
c = int(input()) # intercept of wall with y-axis: a parameter that defines position of the wall
#exception: if the wall is perfectly upright i.e. vertical with respect to the horizontal, this is the intercept with the x axis
alpha = m.atan(Fy/Fx) #angle the force makes with the horizontal
beta = 90*(PI/180) - alpha #angle the wall makes with the horizontal
gamma = m.atan(-1/m.tan(beta)) #angle the normal to the wall makes with the horizontal

In [ ]:
#these lines of code initialize an array of x and y values according to the equation of the normal to the wall
#this line will be the trajectory the end effector will follow
#this is why x values start from Rmin, so that the x,y values remain in workspace
if beta == 90*(PI/180):
  x = np.linspace(Rmin,c,100)
  y = 0*x
else:
  x = np.linspace(Rmin*m.cos(gamma),Rmax*m.cos(gamma),100)
  y = m.tan(gamma)*x + c

In [ ]:
#if the final point in the trajectory is not in workspace it is not possible to apply force
#in this case the code prints an error message and exits
if not isworkspace(x[99],y[99]) or not isworkspace(x[0],y[0]):
  print("wall surface not in workspace")
  sys.exit()

In [ ]:
#animation object for reaching wall (using the previously defined frame method to follow the normal)
wall = animation.FuncAnimation(fig,frame, frames = 100, blit = True)

In [ ]:
wall #viewing the animation object

In [ ]:
#finding joint variables for final point
theta = m.acos((x[99]**2 + y[99]**2 - l1**2 - l2**2)/(2*l1*l2))
q1 = m.atan(y[99]/x[99]) - m.atan(l2*m.sin(theta)/(l1 + l2*m.cos(theta)))
q2 = q1 + theta

In [ ]:
#using the joint variables and input force to find torque applied at joints
cartspace2 = [[-1*l1*m.sin(q1), l1*m.cos(q2)],[-1*l2*m.sin(q2) l2*m.cos(q2)]]
F = [Fx, Fy]
tau = np.matmul(cartspace2,F)
print(tau)

In [ ]:
#task 3
#entering spring constant, equilibrium point, displacement point
k = int(input())
x0 = int(input())
y0 = int(input())
x1 = int(input())
y1 = int(input())

In [ ]:
#finding other extreme
p = x -x0
q = y - y0
x2 = x0 - p
y2 = y0 - q
#checking if both extremes are in workspace before proceeding
if not isworkspace(x1,y1) or not isworkspace(x2,y2):
  print("displacement region not in workspace")
  sys.exit()
# defining x and y arrays that store the values that the end effector will travel through
if x1>=x2:
  x = np.linspace(x2,x1,100)
else:
  x = np.linspace(x1,x2,100)
if y1>=y2:
  y = np.linspace(y2,y1,100)
else:
  y = np.linspace(y1,y2,100)

In [ ]:
#external additional driving forces
FX = int(input())
FY = int(input())
#defining the spring forces, adding with external driving forces
Fx = k*(x-x0) + FX
Fy = k*(y-y0) + FY
#defining array of joint variables that we will go through for each end effector point
theta = np.arccos((x**2 + y**2 - l1**2 - l2**2)/(2*l1*l2))
q1 = np.arctan(y/x) - np.arctan(l2*np.sin(theta)/(l1 + l2*np.cos(theta)))
q2 = q1 + theta

In [ ]:
#defining array of torques we will go through
#reason: spring action can only be simulated accurately if we consider the varying speeds along our trajectory
# - otherwise it's just oscillation around a point
cartspace2 = [[-1*l1*m.sin(q1), l1*m.cos(q2)],[-1*l2*m.sin(q2) l2*m.cos(q2)]]
F = [Fx, Fy]
tau = np.matmul(cartspace2,F)

tau1 = tau[0]
tau2 = tau[1]

#from here onwards, same logic as in task 1 part b
#inital conditions of q and x are changed as they are defined by the user here
t = np.linspace(0,100,100)

#setting initial conditions
XX0 = x1
YY0 = y1
XX = [XX0, YY0]
theta = m.acos((x1**2 + y1**2 - l1**2 - l2**2)/(2*l1*l2))
q10 = m.atan(y1/x1) - m.atan(l2*m.sin(theta)/(l1 + l2*m.cos(theta)))
q20 = q1 + theta
q0 = [q10,q20]
Q10 = 0
Q20 = 0
Q0 = [Q10, Q20]

In [ ]:
Q = odeint(ode2,Q0,t) #solution for angular velocities
q = odeint(ode1,q0,t) #solution for joint variables as a function of time
X = odeint(odexy,XX,t) #solution for x and y as functions of time
x = X[0]
y = X[1]

In [ ]:
spring = animation.FuncAnimation(fig,frame, frames = 100, blit = True) #animation for spring

In [ ]:
spring #viewing animation
#loop option already provided in output - is able to simulated repeated oscillations over time

In [ ]:
# task 4
#initializing joint 1 variable to 35 degrees
q1 = 35*PI/180
#creating empty x and y arrays to store end effector values
X = []
Y = []
#moving joint 1 by 1 degree, and each time moving joint 2 from 35 to 145 degrees (again, 1 degree at a time)
#at each configuration of the robot's arms, we store the end effector coordinates in X,Y
for i in range(111):
  q2 = 35*PI/180
  for j in range(111):
    q2 = q2 + PI/180
    X.append(l1*m.cos(q1) + l2*m.cos(q2))
    Y.append(l1*m.sin(q1) + l2*m.sin(q2))
  q1 = q1 + PI/180
#setting limits for axes so that it's easier to understand the graph
plt.xlim(-(l1 + l2),(l1 + l2))
plt.ylim(0,l1 + 1.1*l2)
#creating a scatterplot of end effector points obtained by above process
#this will give us a "shaded" region that is our workspace
sns.scatterplot(X,Y)
plt.show()